In [2]:
import tensorflow as tf
import tensorflow.keras as keras
from utils import load_class_names, output_boxes, draw_outputs, resize_image
import cv2
import numpy as np
from yolov3 import YOLOv3Net
import math

In [3]:
model_size = (416, 416,3)
num_classes = 4
class_name = './data/coco.names'
max_output_size = 40
max_output_size_per_class= 20
iou_threshold = 0.5
confidence_threshold = 0.5
cfgfile = 'cfg/yolov3.cfg'
weightfile = 'weights/yolov3_weights.tf'
img_path = "data/images/test.jpg"

In [4]:
model = YOLOv3Net(cfgfile,model_size,num_classes)
model.load_weights(weightfile)

In [5]:
print(model.output)

Tensor("concat_6:0", shape=(None, 10647, 9), dtype=float32)


In [9]:
X = np.array([])
Y = np.array([])
counter = 0
for dirname, _, filenames in os.walk('./dataset'):
    for filename in filenames:
        counter = counter + 1
        if counter % 100 == 0:
            print(counter)
        img_file = os.path.join(dirname, filename)
        if '.jpg' in img_file:
            image = keras.preprocessing.image.load_img(img_file, target_size=(416, 416))
            image_arr = keras.preprocessing.image.img_to_array(image)
            if len(X) == 0:
                X = np.array([image_arr])
            else:
                X = np.append(X, [image_arr], axis=0)
                
                
            label_file = img_file[:-4] + '.txt'

            labels = []
            with open(label_file) as f:
                content = f.readlines()
            content = [x.strip() for x in content] 
            for YOLO_SIZE in [13, 26, 52]:
                label = np.zeros((3 * YOLO_SIZE * YOLO_SIZE, 9))

                if YOLO_SIZE == 13:
                    anchors = [(10, 13), (16, 30), (33, 23)]
                elif YOLO_SIZE == 26:
                    anchors = [(30, 61), (62, 45), (59, 119)]
                else:
                    anchors = [(116, 90), (156, 198), (373, 362)]

                for line in content:
                    line_arr = line.split()
                    species = int(line_arr[0])
                    x = float(line_arr[1])
                    y = float(line_arr[2])
                    w = int(float(line_arr[3]) * 416)
                    h = int(float(line_arr[4]) * 416)

                    cell_x = min(int(x * YOLO_SIZE), YOLO_SIZE-1)
                    cell_y = min(int(y * YOLO_SIZE), YOLO_SIZE-1)
                    
                    x = int(x * 416)
                    y = int(y * 416)

                    best_anchor = -1
                    min_iou = 0
                    for i in range(3):
                        overlap = min(anchors[i][0], w) * min(anchors[i][1], h)
                        iou = overlap / (anchors[i][0] * anchors[i][1]  + w * h - overlap)
                        if iou > min_iou:
                            min_iou = iou
                            best_anchor = i

                    label[best_anchor*YOLO_SIZE * YOLO_SIZE + cell_y * YOLO_SIZE + cell_x, :] = [x, y, w, h, 1, 0, 0, 0, 0]
                    label[best_anchor*YOLO_SIZE * YOLO_SIZE + cell_y * YOLO_SIZE + cell_x, 5+species] = 1

                if len(labels) == 0:
                    labels = label
                else:
                    labels = np.concatenate((labels, label))
                    
            if len(Y) == 0:
                Y = np.array([labels])
            else:
                Y = np.append(Y, [labels], axis = 0)

            

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000


In [10]:
print('X shape: ', X.shape)
print('Y shape: ', Y.shape)

X shape:  (1501, 416, 416, 3)
Y shape:  (1501, 10647, 9)


In [20]:
def yolo_loss(y,y_pred):
    loss = tf.square(y[:, :, 4] - y_pred[:, :, 4])
    loss += tf.square(y[:, :, 0] - y_pred[:, :, 0]) * y[:, :, 4]
    loss += tf.square(y[:, :, 1] - y_pred[:, :, 1]) * y[:, :, 4]
    loss += tf.square(tf.sqrt(y[:, :, 2]) - tf.sqrt(y_pred[:, :, 2])) * y[:, :, 4]
    loss += tf.square(tf.sqrt(y[:, :, 3]) - tf.sqrt(y_pred[:, :, 3])) * y[:, :, 4]
    loss += tf.square(y[:, :, 5] - y_pred[:, :, 5]) * y[:, :, 4]
    loss += tf.square(y[:, :, 6] - y_pred[:, :, 6]) * y[:, :, 4]
    loss += tf.square(y[:, :, 7] - y_pred[:, :, 7]) * y[:, :, 4]
    loss += tf.square(y[:, :, 8] - y_pred[:, :, 8]) * y[:, :, 4]
    loss = tf.reduce_mean(keras.backend.sum(loss, axis=1), axis=0)

    return loss

In [21]:
model.compile(
    optimizer=keras.optimizers.Adam(lr=1e-3),
    loss=yolo_loss
)

In [22]:
model.fit(X, Y, batch_size=32, epochs=100)

Epoch 1/100
24/47 [==============>...............] - ETA: 5:28 - loss: 474371252551680.0000

KeyboardInterrupt: 